# Data processing

## Purpose
This notebook just process the raw data obtained from [SSP/SP](http://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx) into two datasets.

# Setup

## Library import
We import all the required Python libraries

In [2]:
import glob
import os
import warnings

from feature_engine.selection import DropConstantFeatures, DropDuplicateFeatures
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
import cufflinks as cf

# Options for libraries
cf.go_offline(connected=True)
cf.set_config_file(theme='white')
pd.options.display.max_columns = None
pd.options.display.max_rows = 30
plotly.offline.init_notebook_mode(connected=True)
plt.style.use('seaborn-paper')
warnings.filterwarnings("ignore")

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

## Custom functions

In [3]:
def create_df_from_data(file_name_pattern: str = 'Mensal-Capital*') -> pd.DataFrame:
    file_list = glob.glob(RAW_DATA_FOLDER + file_name_pattern + '.csv')
    file_list.sort()

    df = pd.DataFrame()

    for file in file_list:
        year = file[-8:-4]
        df_temp = pd.read_csv(file, sep=';', na_values='...', thousands='.')
        df_temp.dropna(how='all', axis=1, inplace=True)
        df_temp.drop(columns=['Total'], inplace=True)
        df_temp.rename(columns={
        'Jan': year+'01', 'Fev': year+'02', 'Mar': year+'03', 'Abr': year+'04', 'Mai': year+'05', 'Jun': year+'06',
        'Jul': year+'07','Ago': year+'08','Set': year+'09','Out': year+'10','Nov': year+'11','Dez': year+'12'}, inplace=True)
        if df.empty:
            df = df_temp.copy()
        else:
            df = df.merge(df_temp, how='left', on='Ocorrencia')

    df = df.pivot_table(columns='Ocorrencia')
    return df

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [4]:
# Data folders
RAW_DATA_FOLDER = "../data/raw/"
INTERIM_DATA_FOLDER = "../data/interim/"
PROCESSED_DATA_FOLDER = "../data/processed/"

RANDOM_STATE = 42


# Data import
We retrieve all the required data for the analysis.

In [5]:
# Mensal-Capital

df_crime = create_df_from_data('Mensal-Capital*')
        
print(f"""Qtd de meses e tipos de crimes: {df_crime.shape}""")
df_crime.head()

Qtd de meses e tipos de crimes: (31, 23)


Ocorrencia,ESTUPRO,ESTUPRO DE VULNERÁVEL,FURTO - OUTROS,FURTO DE VEÍCULO,HOMICÍDIO CULPOSO OUTROS,HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO,HOMICÍDIO DOLOSO (2),HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,LATROCÍNIO,LESÃO CORPORAL CULPOSA - OUTRAS,LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO,LESÃO CORPORAL DOLOSA,LESÃO CORPORAL SEGUIDA DE MORTE,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3),Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,Nº DE VÍTIMAS EM LATROCÍNIO,ROUBO - OUTROS,ROUBO A BANCO,ROUBO DE CARGA,ROUBO DE VEÍCULO,TENTATIVA DE HOMICÍDIO,TOTAL DE ESTUPRO (4),TOTAL DE ROUBO - OUTROS (1)
201901,61,153,17270,2847,3,66,55,0,6,89,1106,2565,2,56,0,6,10672,1,307,1747,71,214,10980
201902,67,119,19468,2866,4,31,48,0,4,98,988,2495,2,48,0,5,9948,1,257,1585,57,186,10206
201903,72,166,23693,3031,0,31,75,0,3,97,1170,2662,0,85,0,3,11509,2,299,1796,64,238,11810
201904,60,169,18730,3274,0,22,59,0,3,172,1255,2793,1,59,0,3,10844,1,296,1865,69,229,11141
201905,58,141,20480,3450,1,47,45,0,7,104,1285,2429,1,50,0,7,11263,1,283,1914,67,199,11547


In [6]:
# ProdutividadePolicial-Capital

df_prod_policial = create_df_from_data(file_name_pattern='ProdutividadePolicial-Capital*')

        
print(f"""Qtd de meses e tipos de ações policiais: {df_prod_policial.shape}""")
df_prod_policial.head()

Qtd de meses e tipos de ações policiais: (31, 13)


Ocorrencia,Nº DE ARMAS DE FOGO APREENDIDAS,Nº DE FLAGRANTES LAVRADOS,Nº DE INFRATORES APREENDIDOS EM FLAGRANTE,Nº DE INFRATORES APREENDIDOS POR MANDADO,Nº DE PESSOAS PRESAS EM FLAGRANTE,Nº DE PESSOAS PRESAS POR MANDADO,Nº DE PRISÕES EFETUADAS,Nº DE VEÍCULOS RECUPERADOS,OCORRÊNCIAS DE APREENSÃO DE ENTORPECENTES(1),OCORRÊNCIAS DE PORTE DE ENTORPECENTES,OCORRÊNCIAS DE PORTE ILEGAL DE ARMA,OCORRÊNCIAS DE TRÁFICO DE ENTORPECENTES,TOT. DE INQUÉRITOS POLICIAIS INSTAURADOS
201901,242,2190,173,77,2647,1345,2537,1828,34,115,95,556,7471
201902,224,2230,248,15,2677,1158,2696,1623,33,133,76,632,7993
201903,266,2566,338,20,3016,1080,3107,1909,29,141,113,703,7771
201904,279,2456,336,38,2996,1301,3072,1954,33,111,108,689,7991
201905,238,2600,308,21,3064,1300,3199,1905,52,153,92,767,8817


In [13]:
df_consolidado = pd.concat([df_prod_policial, df_crime], axis=1)
df_consolidado["ANO"] = df_consolidado.index.str[:4]
df_consolidado["MES"] = df_consolidado.index.str[-2:]

df_consolidado = df_consolidado[[
    'ANO', 'MES', 'Nº DE ARMAS DE FOGO APREENDIDAS', 'Nº DE FLAGRANTES LAVRADOS', 'Nº DE INFRATORES APREENDIDOS EM FLAGRANTE',
    'Nº DE INFRATORES APREENDIDOS POR MANDADO', 'Nº DE PESSOAS PRESAS EM FLAGRANTE', 'Nº DE PESSOAS PRESAS POR MANDADO',
    'Nº DE PRISÕES EFETUADAS', 'Nº DE VEÍCULOS RECUPERADOS', 'OCORRÊNCIAS DE APREENSÃO DE ENTORPECENTES(1)',
    'OCORRÊNCIAS DE PORTE DE ENTORPECENTES', 'OCORRÊNCIAS DE PORTE ILEGAL DE ARMA', 'OCORRÊNCIAS DE TRÁFICO DE ENTORPECENTES',
    'TOT. DE INQUÉRITOS POLICIAIS INSTAURADOS', 'ESTUPRO', 'ESTUPRO DE VULNERÁVEL', 'FURTO - OUTROS', 'FURTO DE VEÍCULO',
    'HOMICÍDIO CULPOSO OUTROS', 'HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO', 'HOMICÍDIO DOLOSO (2)',
    'HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO', 'LATROCÍNIO', 'LESÃO CORPORAL CULPOSA - OUTRAS','LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO',
    'LESÃO CORPORAL DOLOSA', 'LESÃO CORPORAL SEGUIDA DE MORTE', 'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3)',
    'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO', 'Nº DE VÍTIMAS EM LATROCÍNIO', 'ROUBO - OUTROS', 'ROUBO A BANCO',
    'ROUBO DE CARGA', 'ROUBO DE VEÍCULO', 'TENTATIVA DE HOMICÍDIO', 'TOTAL DE ESTUPRO (4)', 'TOTAL DE ROUBO - OUTROS (1)']].copy()

print(f"""Dimensões da df resultante: {df_consolidado.shape}""")
df_consolidado.head()

Dimensões da df resultante: (31, 38)


Ocorrencia,ANO,MES,Nº DE ARMAS DE FOGO APREENDIDAS,Nº DE FLAGRANTES LAVRADOS,Nº DE INFRATORES APREENDIDOS EM FLAGRANTE,Nº DE INFRATORES APREENDIDOS POR MANDADO,Nº DE PESSOAS PRESAS EM FLAGRANTE,Nº DE PESSOAS PRESAS POR MANDADO,Nº DE PRISÕES EFETUADAS,Nº DE VEÍCULOS RECUPERADOS,OCORRÊNCIAS DE APREENSÃO DE ENTORPECENTES(1),OCORRÊNCIAS DE PORTE DE ENTORPECENTES,OCORRÊNCIAS DE PORTE ILEGAL DE ARMA,OCORRÊNCIAS DE TRÁFICO DE ENTORPECENTES,TOT. DE INQUÉRITOS POLICIAIS INSTAURADOS,ESTUPRO,ESTUPRO DE VULNERÁVEL,FURTO - OUTROS,FURTO DE VEÍCULO,HOMICÍDIO CULPOSO OUTROS,HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO,HOMICÍDIO DOLOSO (2),HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,LATROCÍNIO,LESÃO CORPORAL CULPOSA - OUTRAS,LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO,LESÃO CORPORAL DOLOSA,LESÃO CORPORAL SEGUIDA DE MORTE,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3),Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,Nº DE VÍTIMAS EM LATROCÍNIO,ROUBO - OUTROS,ROUBO A BANCO,ROUBO DE CARGA,ROUBO DE VEÍCULO,TENTATIVA DE HOMICÍDIO,TOTAL DE ESTUPRO (4),TOTAL DE ROUBO - OUTROS (1)
201901,2019,01,242,2190,173,77,2647,1345,2537,1828,34,115,95,556,7471,61,153,17270,2847,3,66,55,0,6,89,1106,2565,2,56,0,6,10672,1,307,1747,71,214,10980
201902,2019,02,224,2230,248,15,2677,1158,2696,1623,33,133,76,632,7993,67,119,19468,2866,4,31,48,0,4,98,988,2495,2,48,0,5,9948,1,257,1585,57,186,10206
201903,2019,03,266,2566,338,20,3016,1080,3107,1909,29,141,113,703,7771,72,166,23693,3031,0,31,75,0,3,97,1170,2662,0,85,0,3,11509,2,299,1796,64,238,11810
201904,2019,04,279,2456,336,38,2996,1301,3072,1954,33,111,108,689,7991,60,169,18730,3274,0,22,59,0,3,172,1255,2793,1,59,0,3,10844,1,296,1865,69,229,11141
201905,2019,05,238,2600,308,21,3064,1300,3199,1905,52,153,92,767,8817,58,141,20480,3450,1,47,45,0,7,104,1285,2429,1,50,0,7,11263,1,283,1914,67,199,11547


# Data export

Creating and exporting interim datasets for data viz.

In [14]:
df_crime.to_excel(INTERIM_DATA_FOLDER+'df_crime.xlsx')
df_prod_policial.to_excel(INTERIM_DATA_FOLDER+'df_prod_policial.xlsx')

df_consolidado.to_parquet(INTERIM_DATA_FOLDER+'df_consolidado.pqt')